In [ ]:
# Importing necessary libraries
import pandas as pd
import geopandas as gpd
from sklearn.ensemble import RandomForestClassifier
from textblob import TextBlob
import folium

# Load the datasets
accident_data = pd.read_csv('data/ACCIDENT.csv')
road_surface_data = pd.read_csv('data/ROAD_SURFACE_COND.csv')
atmospheric_data = pd.read_csv('data/ATMOSPHERIC_COND.csv')
vehicle_data = pd.read_csv('data/VEHICLE.csv')
traffic_counts_data = pd.read_csv('data/Traffic_Count_Locations.csv')

# Step 1: Data Preprocessing
print("Step 1: Preprocessing the data...")
# Merge datasets on the accident number
merged_data = pd.merge(accident_data, road_surface_data, on='ACCIDENT_NO')
merged_data = pd.merge(merged_data, atmospheric_data, on='ACCIDENT_NO')
merged_data = pd.merge(merged_data, vehicle_data, on='ACCIDENT_NO')

# Drop any rows with missing values
merged_data.dropna(inplace=True)

# Save the cleaned data to use later
merged_data.to_csv('cleaned_data.csv', index=False)
print("Data preprocessing completed!")

# Step 2: Crash Hotspot Prediction
print("Step 2: Predicting crash hotspots...")

# Selecting features for model
X = merged_data[['TRAFFIC_COUNT', 'ROAD_SURFACE', 'ATMOSPHERIC_COND']]
y = merged_data['SEVERITY']

# Train the model
model = RandomForestClassifier()
model.fit(X, y)

# Load the geographic data for visualizing hotspots
geo_data = gpd.read_file('data/VICTORIAN_ROAD_CRASH_DATA.geojson')

# Predict crash hotspots
geo_data['predicted_risk'] = model.predict(geo_data[['TRAFFIC_COUNT', 'ROAD_SURFACE', 'ATMOSPHERIC_COND']])

# Step 3: Visualizing Hotspots using Folium
print("Step 3: Visualizing crash hotspots on the map...")

# Create a map centered on Victoria
map_victoria = folium.Map(location=[-37.4713, 144.7852], zoom_start=7)

# Add accident locations with high risk to the map
for _, row in geo_data.iterrows():
    if row['predicted_risk'] > 0.7:
        folium.Marker(
            location=[row['geometry'].y, row['geometry'].x],
            popup=f"Risk Level: {row['predicted_risk']}",
            icon=folium.Icon(color="red")
        ).add_to(map_victoria)

# Save the map as an HTML file to view later
map_victoria.save('crash_hotspots_map.html')
print("Hotspot map created and saved as crash_hotspots_map.html")

# Step 4: Generate Executive Brief
print("Step 4: Creating the executive brief...")

def create_executive_brief(model_output):
    summary = f"Based on the analysis, {model_output['hi
